In [43]:
import os
import time
import serial
import numpy as np

In [44]:
def request_data(msg, msgType = str):

    data = msgType(input(msg))
    return data

In [46]:
# Función para guardar datos en archivo
def save_data(data, file_path):

    with open(file_path, "w") as file:

        for key, value in data.items():
            
            "Timestamp : Value"
            file.write(f'{key} : {value}\n')

In [47]:
def getOption():

    print("--- --- ---")
    print("1. New Register")
    print("0. End Test")
    print("--- --- --- \n")
    option = int(input("Select an option"))
    
    return option

In [48]:
def read_serial_data(command, serial_port):
    
    serial_port.write(command.encode())
    sensor_data = {}  # Diccionario para almacenar los datos con sus timestamps

    # Esperar a que comiencen los datos
    while serial_port.in_waiting == 0:
        pass

    start_time = time.time()  # Registra el tiempo de inicio

    while True:
        current_time = time.time()  # Tiempo actual

        # Verificar si han pasado 3 minutos
        if (current_time - start_time) >= 180:
            break

        if serial_port.in_waiting > 0:
            data = serial_port.readline().decode('utf-8').strip()

            try:

                timestamp, value = map(int, data.split(','))

                sensor_data[timestamp] = value
                print("Time(ms): ", timestamp, " --- ", "Value(mv):", value)

            except ValueError:

                print(f"Datos recibidos no válidos o incompletos: {data}")
                break

    return sensor_data


In [ ]:
if __name__ == "__main__":

    # --- Configurando serial ---
    port = "COM4"
    baudRate = 115200
    serialPort = serial.Serial(port, baudRate)
    time.sleep(10)

    option = getOption()

while option == 1:

    # Creando estructura para datos del usuario
    user_data = {       
    "name": request_data("Name(s): "),
    "last_name": request_data("Last name(s): "),
    "age": request_data("Age: ", int),
    "gender": request_data("Gender: "),
    "weight": request_data("Weight (kg): ", float),
    "height": request_data("Height (centimeters): ", float),
    "systolic_pressure": request_data("Systolic Pressure: ", int),
    "diastolic_pressure": request_data("Diastolic Pressure: ", int),
    "hypoglycemia": request_data("Do you suffer from hypoglycemia? (yes/no): "),
    "diabetes": request_data("Do you suffer from diabetes? (yes/no): "),
    "fasting": request_data("Did you fast? (yes/no): ")
    }

    # Creando directorio para el usuario
    path = "../Data"
    folder_id = f"{user_data['last_name']}_{user_data['name']}"
    folder_path = os.path.join(path, folder_id)
    os.makedirs(folder_path, exist_ok=True)

    # Creando subdirectorio 'Private'
    private_folder_path = os.path.join(folder_path, "Private")
    os.makedirs(private_folder_path, exist_ok=True)

    # Guardando datos generales
    save_data(user_data, os.path.join(private_folder_path, "user_data.txt"))

    # Creando subdirectorios para PPG y ECG
    ppg_folder_path = os.path.join(folder_path, "PPG")
    ecg_folder_path = os.path.join(folder_path, "ECG")
    os.makedirs(ppg_folder_path, exist_ok=True)
    os.makedirs(ecg_folder_path, exist_ok=True)

    # Realización de las pruebas
    for test_id in range(1, 3):
        
        print(f"--- Test {test_id} ---")
    
        glucose_value = int(input("Take the blood glucose sample and enter the obtained value: "))

        # Obtencion de señales ECG y PPG
        print("Toma la prueba de ECG")
        time.sleep(5)

        # Guardando datos de ECG
        ECG_data = read_serial_data("ECG", serialPort)
        save_data(ECG_data, os.path.join(ecg_folder_path, f"ECG_V{test_id}.txt"))

        print("Toma la prueba de PPG")
        time.sleep(10)

        # Guardando datos de PPG
        PPG_data = read_serial_data("PPG", serialPort)
        save_data(PPG_data, os.path.join(ppg_folder_path, f"PPG_V{test_id}.txt"))

        ECG_data.clear()
        PPG_data.clear()

        print("End Test, Choose a new option: ")

    option = getOption()